In [1]:
!pip3 uninstall twint --yes


Found existing installation: twint 2.1.21
Uninstalling twint-2.1.21:
  Successfully uninstalled twint-2.1.21


In [2]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to c:\users\reema\appdata\local\temp\pip-req-build-j4ii6zj7
  Created wheel for twint: filename=twint-2.1.21-py3-none-any.whl size=39138 sha256=771c60786cf35a511cf0f6de894fbd0b47a7ba4815f585687d1bf1f1dc928684
  Stored in directory: C:\Users\reema\AppData\Local\Temp\pip-ephem-wheel-cache-58lyhfli\wheels\c9\d4\7d\fb0732852f335c92c52450e5c86b6ba5fb09fa226f3584db34
Successfully built twint


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\users\reema\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip3 install nest_asyncio

You should consider upgrading via the 'c:\users\reema\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip3 install python-dotenv

You should consider upgrading via the 'c:\users\reema\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip3 freeze >requirements.txt

In [6]:
!pip3 install tweepy

You should consider upgrading via the 'c:\users\reema\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [62]:
'''
Author: Paul Owe
Description: This is a twitter client that handles authentication to twitter via tweepy API.
For authentication you will need API keys from twitter. You can apply here https://developer.twitter.com/en
Note: Keep these keys private.
Last updated: Mar 4, 2021
/'''
# import os
# import sys
# from tweepy import API
# from tweepy import OAuthHandler
# from dotenv import load_dotenv

# def get_twitter_auth():
#     #set up twitter authentication

#     # Return: tweepy.OAuthHandler object

#     try:
#         load_dotenv()
#         access_secret = "gjxpdHcVwVT4G7aGBMz654DLHk1aVu1zBXVTohMifq2XU"
#         access_token = "922977192-jzCCOIHFYh8Fs9N5DjlxZfW6CxgtLzPZyohYuqTX"
#         consumer_secret = "pCmBTZ6dBqCaf1BTffkF8il0AAiWIhgUlYhhZ7jDZFMFuDyNa5"
#         consumer_key = "bPIKaiR4d91amYlft7s6p39Nz"
#     except KeyError:
#         sys.stderr.write("TWITTER_* environment variables not set\n")
#         sys.exit(1)
#     auth = OAuthHandler(consumer_key, consumer_secret)
#     auth.set_access_token(access_token, access_secret)
#     return auth


# def get_twitter_client():
#     #Setu twitter API client.

#     # Return tweepy.API object

# #     auth = get_twitter_auth()
#     client = API(auth)
#     return client

In [68]:
import os, sys, time, csv, math
# Basic utilities for processing data in python
import pandas as pd
import numpy as np
# Open source Twitter intelligence tool
import twint
# Twitter API client
from twitter_client import get_twitter_client

import tweepy
from tweepy import Cursor
# Removes runtimeError in jupyterlab -> This event loop is already running
import nest_asyncio
nest_asyncio.apply()

In [69]:

%%capture
# Configure twint query parameters to find all users talking about FSL

qp = twint.Config()
qp.Search = "Consortium of Online International Learning"
qp.Output = "vm-terms.csv"
qp.Store_csv = True

# Run twint on configured qp

twint.run.Search(qp)
print("Running search query")

In [70]:
# variables
MAX_FRIENDS = 15000
client = get_twitter_client()
max_pages = math.ceil(MAX_FRIENDS/ 5000)

Traceback (most recent call last):
  File "C:\Users\reema\twitter_client.py", line 21, in get_twitter_auth
    consumer_key = os.environ['TWITTER_CONSUMER_KEY']
  File "c:\users\reema\appdata\local\programs\python\python36\lib\os.py", line 669, in __getitem__
    raise KeyError(key) from None
KeyError: 'TWITTER_CONSUMER_KEY'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\reema\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-70-8cdba885f4ec>", line 3, in <module>
    client = get_twitter_client()
  File "C:\Users\reema\twitter_client.py", line 38, in get_twitter_client
    auth = get_twitter_auth()
  File "C:\Users\reema\twitter_client.py", line 27, in get_twitter_auth
    sys.exit(1)
SystemExit: 1

During handling of the above exception, another exception occurred:

Trac

TWITTER_* environment variables not set


TypeError: object of type 'NoneType' has no len()

In [73]:
#setup paginate function
def paginate(items, n):
    """
    Generate n-sized chunks from Items

    """
    for i in range(0, len(items), n):
        yield items[i:i+n]

In [75]:
#Open csv file to extract column names
with open('vm-terms.csv', encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file)
    columnNames = []
    for row in csv_reader:
        columnNames.append(row)
        #break after reading first row
        break
print(columnNames)
columnNames = columnNames[0]

[['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video', 'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']]


In [80]:
data = pd.read_csv('vm-terms.csv', names=columnNames)
#extract user ids
# userids = data.id.tolist()
# users_list = []
# for id in userids:
#     try:
#         print(id)
#         users_list.append(client.get_user(user_id=id))
#     except tweepy.TweepError as err:
#         print("user not found")
#         pass
# print(users_list[1])



c:\users\reema\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1,5,6,15,16,17,21,22,23,25,26,27,28,29,30,32,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [81]:
#extract list of username from the csv file
usernames = data.username.tolist()

In [82]:
print(usernames)

['username', 'ipminow', 'jobacle', 'mapbroward', '_icecreamcake__', '_smartageing_', 'aphsa1', 'oia_usk', 'messybedroom', 'taghouston', 'hopkinsamp', 'informationday2', 'thalesmea', 'meeramimie', 'fromgen2gen', 'youngerandfever', 'ccs_detroit', 'possabilityppl', 'melissasmiltos', 'possabilityppl', 'possabilityppl', 'asciiqwerty', 'tacomatopia', 'theoncopt', 'hypori_vm', 'coreresponse_', 'ltadpartners', 'hypori_vm', 'possabilityppl', 'itstiatime00', 'possabilityppl', 'afrotech', 'gboeyens', 'possabilityppl', 'miamifoundation', 'donalconlon', 'hopkinsamp', 'aeconservatoire', 'the_acu', 'weynatten', 'cuhkglobal', 'steeperclinic', 'tommers71', 'hypori_vm', 'plusrelocation', 'aphsa1', 'motusdotcom', 'ccbatexas', 'climatechance', 'abruzzoconnect', 'hopkinsamp', 'climatechance', 'timestechbuzz', 'firenice7700', 'climatechance', 'ccit_fraunhofer', 'aheadp_ship', 'vishall30067075', 'kay2531388510', 'arts4allflorida', 'foreignpolicywd', 'techjasond', 'canada4news', 'newsroomgc', 'newmobilitymag'

In [87]:
# make sure you are not repeating already scraped users. You will waste time
# for this, check your network-progress list for users you already have scraped and remove them from usernames 

# load updated username list before you continue to build your network 

progress_params = ['explored_users', 'skipped_users']
network_progress = pd.read_csv('vm-progress.csv', names=progress_params)
explored_users = network_progress.explored_users.tolist()
skipped_users = network_progress.skipped_users.tolist()
# ignoring the nan's and column names, len(explored) + len(skipped) = len(usernames) 
unames = [ x for x in usernames if x not in (explored_users + skipped_users)]

#loop through all users and write their followers to csv
usernames=['1paulowe']
for user in usernames:

#     try:
    fname = "vm-edges-2.csv"
    fieldnames = ['Source', 'Target']
    with open(fname, 'a') as f:

        csv_writer = csv.DictWriter(f, fieldnames=fieldnames)

        # get a user's followers list
        for followers in Cursor(client.followers_ids, screen_name=user).pages(max_pages):
            for chunk in paginate(followers, 100):
                userFollowers = client.lookup_users(user_ids=chunk)

                for follower in userFollowers:

                    #print(len(users))

                    #print(follower.screen_name)
                    print(follower.screen_name)
                    csv_writer.writerow({'Source': follower.screen_name, 'Target': user})

                # f.write(json.dumps(user._json)+"\n")
            if len(followers) == 5000:
                print("More results available. Sleeping for 60 seconds to avoid rate limit")
                time.sleep(60)

        # get a user's following list
        for friends in Cursor(client.friends_ids, screen_name=user).pages(max_pages):

            for chunk in paginate(friends, 100):
                userFollowing = client.lookup_users(user_ids=chunk)
                for followed in userFollowing:
                    csv_writer.writerow({'Source': user, 'Target': followed.screen_name})
            if len(friends) == 5000:
                print("More results available. Sleeping for 60 seconds to avoid rate limit")
                time.sleep(60)

        f.close()

        # Update network-progress
        with open('vm-progress.csv', 'a') as nprog:
            nprog_updater = csv.DictWriter(nprog, fieldnames=progress_params)
            nprog_updater.writerow({'explored_users': user, 'skipped_users': ' '})
            nprog.close()



            
#     except tweepy.TweepError as err:
#         print("Tweepy error occured: {}. Skipped adding user: {}".format(err, user))


#         with open('vm-progress.csv', 'a') as nprog:
#             nprog_updater = csv.DictWriter(nprog, fieldnames=progress_params)
#             nprog_updater.writerow({'explored_users': ' ', 'skipped_users': user})
#             nprog.close()
#         pass

TweepError: [{'code': 17, 'message': 'No user matches for specified terms.'}]